In [1]:
import numpy as np

In [2]:
import ansys.fluent.core as f1

In [3]:
# Launch Fluent solver
solver = f1.launch_fluent(precision = "double",processor_count=2,mode="solver",show_gui=True,version="2d")

In [10]:
# Lists to store simulation results
den = []     #define Density
K = []       #define Thermal Conductivity

i = 2
# Loop for multiple simulations
while i< 3:
    # Read mesh file
    solver.file.read_mesh(file_name="sav{}.msh".format(i))
    
    # Set operating conditions and enable energy model
    solver.tui.define.operating_conditions.gravity("yes")
    solver.tui.define.operating_conditions.gravity("yes", "0", "-9.81")
    solver.setup.models.energy.enabled = True
    
    # Copy material
    solver.execute_tui(r'''/define/materials/copy solid wood ''')
    
    
    # Define and apply material
    solver.tui.define.materials.change_create()
    solver.execute_tui(r'''/define/materials/change-create air air no no yes constant 0.026 no no no no ''')
    solver.execute_tui(r'''/define/materials/change-create wood matrix yes constant 915 yes constant 746 yes constant 0.2 ''')
    solver.execute_tui(r'''/define/materials/change-create wood matrix2 yes constant 2210 yes constant 746 yes constant 0.1 ''')
    
    solver.setup.materials.fluid()

    # Set cell zone conditions
    solver.setup.cell_zone_conditions.solid()
    solver.tui.define.boundary_conditions.zone_type("domain", "solid")
    solver.setup.cell_zone_conditions.solid['domain'].material = 'matrix'
    
    #solver.tui.define.boundary_conditions.zone_type("solid-sav1_surface", "solid")
    #solver.setup.cell_zone_conditions.solid['solid-sav1_surface'].material = 'matrix2'
    solver.setup.cell_zone_conditions.solid['solid-rad_0.02_{}_surface'.format(i)] = {"material" : "matrix2"}
    
    # Modify zone types for boundary conditions
    solver.execute_tui(r'''/define/boundary-conditions/modify-zones/zone-type high wall ''')
    solver.execute_tui(r'''/define/boundary-conditions/modify-zones/zone-type low wall ''')

    # Set temperature conditions
    solver.setup.boundary_conditions.wall['low'](thermal_bc = 'Temperature', t = 300)
    solver.setup.boundary_conditions.wall['low'] = {"material" : "matrix"}
    
    solver.setup.boundary_conditions.wall['high'](thermal_bc = 'Heat Flux',q = 10000)
    
    solver.setup.boundary_conditions.wall['high'] = {"material" : "matrix"}
    
    solver.setup.boundary_conditions.wall['wall1'](thermal_bc = 'Heat Flux')
    solver.setup.boundary_conditions.wall['wall1'] = {"material" : "matrix"}
    solver.setup.boundary_conditions.wall['wall2'](thermal_bc = 'Heat Flux')
    solver.setup.boundary_conditions.wall['wall2'] = {"material" : "matrix"}

    # Initialize and run simulation
    solver.solution.initialization.hybrid_initialize()
    solver.solution.run_calculation.iter_count = 1000
    solver.solution.run_calculation.iterate()

    # Create temperature contour and display it
    solver.results.graphics.contour.create("temperature_contour1")
    solver.results.graphics.contour()
    solver.results.graphics.contour["temperature_contour1"](field = "total-temperature")
    solver.results.graphics.contour["temperature_contour1"].display()

#     solver.results.graphics.picture.save_picture(file_name= "Temperature_sample2")
#     from matplotlib import pyplot as plt
#     from matplotlib import image
#     plt.imshow(image.imread("Temperature_sample2.png"))
   
    # Define and save surface reports
    solver.solution.report_definitions.surface["high"] = {}
    solver.solution.report_definitions.surface['high'].report_type = "surface-areaavg"
    solver.solution.report_definitions.surface['high'] = {"field" : "temperature", "surface_names" : ["high"]}
    solver.solution.report_definitions.surface["low"] = {}
    solver.solution.report_definitions.surface['low'].report_type = "surface-areaavg"
    solver.solution.report_definitions.surface['low'] = {"field" : "temperature", "surface_names" : ["low"]}
    solver.solution.report_definitions.surface["heat_flux"] = {}
    solver.solution.report_definitions.surface['heat_flux'].report_type = "surface-areaavg"
    solver.solution.report_definitions.surface['heat_flux'] = {"field" : "heat-flux", "surface_names" : ["high"]}
    solver.solution.report_definitions.volume["total-volume"] = {}
    solver.solution.report_definitions.volume['total-volume'].report_type = "volume-zonevol"
    solver.solution.report_definitions.volume['total-volume'] = {"zone_names" : ["domain"]}
    
    solver.solution.report_definitions.volume["total-mass"] = {}
    solver.solution.report_definitions.volume['total-mass'].report_type = "volume-mass"
    solver.solution.report_definitions.volume['total-mass'] = {"zone_names" : ["domain"]}
    solver.solution.report_definitions.single_val_expression["k-eff"] = {}
    solver.solution.report_definitions.single_val_expression['k-eff'] = {"define" : "({heat_flux}/(({high}-{low})/0.001))"}
    solver.solution.report_definitions.single_val_expression["bulk-density"] = {}
    solver.solution.report_definitions.single_val_expression['bulk-density'] = {"define" : "({total-mass}/{total-volume})"}
    
    
    
    
    # Define and save named expressions
    
    
    solver.setup.named_expressions["keff"] = {}
    solver.setup.named_expressions['keff'] = {"definition" : "(heat_flux/((high-low)/.001))", "output_parameter" : True}
    solver.execute_tui(r'''/define/parameters/output-parameters/write-to-file "keff" "k_0.01.out" ''')
    
    solver.setup.named_expressions["density"] = {}
    solver.setup.named_expressions['density'] = {"definition" : "({total-mass}/{total-volume})", "output_parameter" : True}
    solver.execute_tui(r'''/define/parameters/output-parameters/write-to-file "density" "den_0.01.out" ''')
    i = i+1
    
    
    


Multicore SMT processors detected. Processor affinity set!
Reading "sav2.msh"...
Buffering for file scan...


       Clearing partially read grid.
   10670 nodes, binary.
     400 nodes, binary.
   19303 2D interior faces, zone  1, binary.
    1777 2D interior faces, zone  2, binary.
     520 2D interior faces, zone  3, binary.
     100 2D wall faces, zone  8, binary.
     100 2D wall faces, zone  9, binary.
     100 2D wall faces, zone 10, binary.
     100 2D wall faces, zone 11, binary.
    9897 mixed cells, zone  7, binary.
    1034 mixed cells, zone  4, binary.

Building...
     mesh
	auto partitioning mesh by Metis (fast),
	distributing mesh
		parts..,
		faces..,
		nodes..,
		cells..,
        bandwidth reduction using Reverse Cuthill-McKee: 4963/67 = 74.0746
     materials,
     interface,
     domains,
     zones,
	low
	high
	wall2
	wall1
	domain
	solid-rad_0.02_2_surface
	rad_0.02_2_surface
	interior-rad_0.02_2_surface
	interior-domain
     parallel,
Done.
The following solver s

AttributeError: report_values is not an allowed Settings objects name.
The most similar names are: report.

/define/parameters/output-parameters/write-to-file "keff" "result_k_eff_9.out" /define/parameters/output-parameters/write-to-file "density" "result_k_eff_9.out" 
Error: No surfaces or cell zones are selected to export.

Error Object: #f
adjoint/                parallel/               server/
define/                 parametric-study/       solve/
display/                plot/                   surface/
exit                    preferences/            views/
file/                   print-license-usage
mesh/                   report/

> 
/solve> animate/                initialize/             report-definitions/
cell-registers/         iterate                 report-files/
convergence-conditions/ monitors/               report-plots/
execute-commands/       patch                   set/

/solve> 
/solve/report-definitions> add                     delete                  list
compute                 delete-all              list-properties
copy                    edit                    renam